# Práctica 1: Análisis exploratorio de datos, preprocesamiento y validación de modelos de clasificación\*

### Minería de Datos: Curso académico 2020-2021

### Profesorado:

* Juan Carlos Alfaro Jiménez
* José Antonio Gámez Martín

### Alumnos:

* Sara Lopez Matilla
* Carlos Morote García

\* Adaptado de las prácticas de Jacinto Arias Martínez y Enrique González Rodrigo

## Introducción

En esta libreta se va ha realizar un estudio de dos conjuntos de datos `pima-indians-diabetes` y `breast-cancer-wisconsin-data`. Se van a trabajar algunos de los aspectos más importantes del proceso KDD (Knowledge Discovery from Data).
El índice que se va a llevar a cabo para conjunto de datos es el siguiente:

* 1: Almacenamiento y carga de datos
* 2: Análisis exploratorio de datos
* 3: Preprocesamiento de datos
* 4: Validación de modelos de clasificación

Sin embargo, antes hay que realizar una preparación del entorno.


## 0. Preparación del entorno

Antes de entrar en materia con los datos es necesario disponer de las herramientas oportunas para hacerlo. En eso es lo que se puede encontrar en este apartado, siendo la parte más fundamental la de importar todos los paquetes de Python que se van a usar.

In [ ]:
# Data manipulation
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Visualization
import seaborn as sns

# Preprocessing 
from sklearn.preprocessing import KBinsDiscretizer, OneHotEncoder
from imblearn import FunctionSampler
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.impute import SimpleImputer

# Modelos
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from imblearn.pipeline import make_pipeline

# Scripts
import p1_utilidades as utils

Con el objeto de que los experimentos sean reproducibles se ha establecido una semilla para la generación de números aleatorios.

In [ ]:
seed = 1234567890

## Pima Indians Diabetes Database

### Acceso y almacenamiento de datos

El conjunto de datos que se va a analizar es `pima_diabetes`. Para poder analizar correctamente este conjunto de datos, es necesario conocerlo.

El objetivo de este conjunto de datos es predecir si un paciente va a tener o no diabetes. Todos los pacientes son mujeres, de al menos 20 años y pertenecen al pueblo pima, un grupo indígena que vive en el estado de Arizona (Estados Unidos). Contiene 768 muestras.

Este conjunto contiene ocho variables predictoras y una variable objetivo.

La variable objetivo es `Outcome`. Tiene dos posibles valores: 0 significa que no se tiene diabetes y 1 significa que sí se tiene diabetes.

Para predecir la variable objetivo, hay ocho variables predictoras, que son las siguientes:

* `Pregnancies`: numero de veces que ha estado embarazada.
* `Glucose`: concentración de la glucosa en dos horas en un test de tolerancia oral a la glucosa.
* `BloodPressure`: presión arterial diastólica (mmHg).
* `SkinThickness`: grosor del pliegue de la piel del tríceps (mm).
* `Insulin`: insulina sérica en dos horas ($\mu$U/ml).
* `BMI`: índice de masa corporal (peso(kg)/(altura(m))^2).
* `DiabetesPedigreeFunction`: función de pedigrí de la diabetes. Esta función analiza las relaciones genealógicas de un ser vivo en el contexto de determinar cómo una cierta característica o fenotipo se hereda y manifiesta, en este caso, la diabetes.
* `Age`: edad de la paciente.

Ahora vamos a cargar el conjunto de datos para poder analizarlo de una forma más exhaustiva posteriormente. Se debe poner el parámetro `index` a `None` debido a que esta base de datos no tiene una columna índice.

In [ ]:
filepath = "../input/pima-indians-diabetes-database/diabetes.csv"

index = None
target = "Outcome"

data = utils.load_data(filepath, index, target)

Primero procedemos a partir el conjunto de datos en conjunto de datos de entrenamiento y conjunto de datos de test. Estos dos subconjuntos que se obtienen se subdividen a su vez en X, siendo las variables predictoras, y en Y, siendo la variable objetivo. Se ha optado por dividir el conjunto de datos en un 70% para el conjunto de entrenamiento y un 30% para el conjunto de test.

Debido a que en ocasiones se utilizará el conjunto de entrenamiento y de test sin distinguir entre variables predictoras y objetivo, y en otras ocasiones se distinguirá, se van a obtener los conjuntos dividos y sin dividir.

Se va a aplicar un holdout estratificado para conservar la proporción de personas que tienen diabetes tanto en el conjunto de entrenamiento como en el conjunto de test. Las instancias también está aleatorizadas. La aplicación de la estratificación y la aleatorización se realiza dentro del método `split_data` de `utils`.

In [ ]:
train_size = 0.7

(train, tests, X_train, X_test, y_train, y_test) = utils.split_data(data, target, seed, train_size)

Ahora podemos observar una muestra del conjunto de entrenamiento. Primero, vamos a observar las variables predictoras y después la variable objetivo. Finalmente, observaremos una muestra sin separar las variables.

In [ ]:
X_train.sample(5, random_state=seed)

In [ ]:
y_train.sample(5, random_state=seed)

In [ ]:
train.sample(5, random_state=seed)

Como se puede intuir, no se va a observar ninguna muestra del conjunto de test, debido a que se podría obtener información que lleve al sobreajuste.

### Análisis exploratorio de datos

Para poder observar los datos, se van a presentar mediante diversas gráficas. El tipo de gráfica dependerá del tipo de información que se esté mostrando.

---

#### Descripción del conjunto de datos

Primero, vamos a obtener el número de instancias que han caído en el conjunto de entrenamiento y el número de variables que tenemos, ya sean variables predictoras u objetivo.

In [ ]:
train.shape

Podemos observar que hemos obtenido 537 instancias de las 768 totales y como ya se había comentado antes, tenemos 9 variables, siendo 8 variables predictoras y 1 la variable clase.

Ahora necesitamos saber de qué tipo son las diferentes variables:

In [ ]:
train.info(memory_usage=False)

De estos datos se obtiene que dos variables predictoras, `BMI` y `DiabetesPedigreeFunction` son de tipo real (`float64`), mientras que todas las demás variables predictoras son de tipo entero (`int64`). La variable clase (`Outcome`) es de tipo categórica. También se observa que no hay contenido nulo en ninguna de las variables.

La variable clase puede tomar distintos valores, que son:

In [ ]:
y_train.cat.categories

Por un lado, puede obtenerse el valor 0, que significa que no se tiene diabetes, y por otro lado, puede obtenerse el valor 1, que significa que se tiene diabetes, como ya se había indicado anteriormente.

También se puede observar que internamente esta variable es de tipo entero, ya que el 0 y el 1 son números enteros.

#### Visualización de las variables

Antes de empezar a visualizar las distintas clases, vamos a comprobar si el conjunto de datos está balanceado.

In [ ]:
utils.plot_barplot(train)

Podemos observar que esta muestra no está balanceada. Tenemos alrededor de un 65% de la población que no tiene diabetes, frente a un 35% que sí tiene.

Primero vamos a realizar el análisis univariado, debido a que este conjunto de datos no tiene un número de variables demasiado extenso.

In [ ]:
utils.plot_histogram(train)

Vamos a comentar cada variable una a una, observando su distribución, los outliers y los posibles valores perdidos:

* `Pregnancies`: Podemos observar que el valor que más se repite es que se ha tenido un hijo, con 99 valores. También podemos observar un valor fuera de lo común, una mujer con 17 mujeres. Es posible que este valor sea considerado un outlier. Descartando el valor 0, podemos observar que esta gráfica muestra una tendencia descendiente, concretamente una distribución geométrica.
* `Glucose`: Podemos observar que esta gráfica está representada mediante una mixtura de distribuciones normales. Además, se puede observar unos valores 0 que no parecen concordar, debido a que el nivel de glucosa no puede ser 0. Este valor se debe a que se ha representado los valores perdidos con un 0.
* `BloodPressure`: Podemos observar que esta gráfica representa una distribución normal. Podemos observar un valor entre 120 y 124, que probablemente sea un outlier. También observamos la presencia del valor 0, que es imposible, ya que es imposible una presión arterial de 0. En este caso, también será la representación de los valores perdidos.
* `SkinThickness`: Podemos observar que la distribución de esta variable es una normal. Volvemos a encontrar el valor 0, que vuelve a corresponderse con los valores perdidos. Sin embargo, en este caso observamos la gran cantidad de ejemplos que tienen este valor, siendo posible que no nos facilite información suficiente esta variable y sea necesario eliminarla.
* `Insulin`: Podemos observar que volvemos a obtener el valor 0 que no debería estar, representado los valores perdidos. Podemos observar el gran número de ejemplos que no tienen esta variable. Es probable que esta variable deba ser eliminada por esta gran cantidad. Apartando estos valores, podemos observar que la distribución de estos datos no está clara, siendo una geométrica o una normal. También se puede observar tres valores a partir de 680 muy aislados, que probablemente sean outliers.
* `BMI`: Podemos observar en esta gráfica una distribución normal. Volvemos a observar valores perdidos. También podemos observar un caso aislado en 67, que probablemente sea un outlier.
* `DiabetesPedigreeFunction`: Podemos observar que los valores que se obtienen un mayor número de veces están entre 0.2 y 0.229, ambos inclusives. En cuanto a la distribución, podría tratarse de una distribución normal o de una distribución geométrica, no se puede obtener solo visualizando la gráfica. En este caso, el valor 0 sí tiene sentido, por lo que no representaría valores perdidos. También podemos observar unos posibles valores de outliers a partir de 1.5.
* `Age`: Podemos observar una distribución descendiente, probablemente una geométrica, excepto por los valores de 20 a 21. El valor entre 72 y 73 años podría tratarse de un outlier. Podemos observar que no hay valores perdidos en esta variable.

Ahora vamos a representar las variables predictoras de manera individual contra la variable a predecir usando histogramas. Con esta representación se pretende observar el poder predictivo de cada variable. Esto se puede observar fácilmente de una forma visual, porque las gráficas que tengan mayor área superpuesta entre las dos variables de la clase objetivo serán las que menor información aporten a la hora de discretizar y viceversa, cuanta menor área superpuesta, mayor poder predictivo.

In [ ]:
utils.plot_dropdown_distplot_ly(train, y_train)

Podemos observar que todas las variables se superponen y por tanto, por sí solas no tendrán un alto poder predictivo. Por tanto, será necesaria la combinación de varias variables para poder realizar una predicción.

---

Se va a observar los outliers que tiene cada variable. Para ello se van a mostrar con gráficas de cajas y bigotes, que nos permiten cómodamente ver los outliers.

In [ ]:
utils.plot_dropdown_boxplot_ly(train)

Se puede observar que la variable que más outliers tiene es `Insulin`. `DiabetesPedigreeFunction` también tiene una gran cantidad de outliers. El tratamiento de los outliers se realizará en el preprocesamiento de los datos, será en ese momento cuando será necesario manejarlos.

<br>
Vamos a observar estadísticamente las diferentes variables. Primero se van a observar las variables numéricas, que son las variables predictoras. Después se van a observar las variables categóricas, que solo es la variable objetivo.


In [ ]:
train.describe(include="number")

In [ ]:
train.describe(include="category")

Antes de realizar el análisis bivariado, sería interesante observar qué variables se pueden eliminar debido a su gran cantidad de valores perdidos, porque si vamos a eliminar estas variables, no tiene demasiado sentido seguir analizándolas. Se va a considerar que una variable no aporta suficiente información, y por tanto se puede eliminar, cuando más del 20% de los valores de dicha variable sean perdidos.

Para saber qué variables cumplen la condición de aportar poco información, se va a usar una función que devuelve los nombres de las variables que tienen más de un 20% de valores perdidos.

Como hemos dicho anteriormente, las variables que tiene valores perdidos son `Glucose`, `BloodPressure`, `SkinThickness`, `Insulin` y `BMI`. Por tanto, solo habrá que comprobar estas variables.

In [ ]:
utils.find_missing_values(0, train[['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']])

Por tanto, solo debemos eliminar las variables `SkinThickness` y `Insulin`.

<br>

Ahora que hemos obtenido las variables que se van a eliminar, podemos realizar una visualización bivariada.

En este caso, al tener 6 variables, se ha optado por usar el Scatter Plot de `graph_objs` en vez del de `express`, debido a que los puntos se mostraban más grandes y era más difícil distinguir los diferentes puntos.

In [ ]:
show = train[['Pregnancies','Glucose', 'BloodPressure', 'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome']]
utils.plot_pairplot_go(show, target='Outcome')

En este análisis bivariado tampoco se puede observar una clara separación de la variable a predecir.

<br>

Para observar la correlación de las variables, se va a utilizar una matriz de confusión que nos indique las correlaciones

In [ ]:
utils.corr_heatmap_ly(show, height=800)

Con esta matriz de confusión se puede observar que las variables no tienen mucha correlación entre ellas dos a dos, de hecho, las variables que más correlación tienen son `Age` y `Pregnancies` con un 54%.

### Preprocesamiento de datos

Antes de inicializar el pipeline, es necesario hacer un preprocesamiento de los datos. En este caso se va a realizar selección de variables, eliminación de outliers, discretización de variables, imputación de valores perdidos, y codificación.

#### Selección de variables

Como hemos señalado en el apartado anterior, hay variables que no aportan ningún poder predictivo debido a la cantidad de valores perdidos que tienen. Estas variables son `SkinThickness` y `Insulin`. Por tanto, estas variables no se utilizarán para calcular un modelo.

En este caso no es necesario eliminarlas del conjunto de datos mediante un transformador, debido a que para la imputación de valores perdidos se va a utilizar `make_column_selector`, que elimina los atributos que no se le pasan en los parámetros. Por tanto, al crear el imputador no se le pasará `SkinThickness` y `Insulin`.

#### Imputación de valores perdidos

Para imputar los valores perdidos, es necesario tener en cuenta cómo se van a imputar los datos. Hay que imputar datos de tres variables: `Glucose`, `BloodPresure` y `BMI`. La variable `BMI`, al ser de tipo flotante, se va a imputar por la media, mientras que las variables `Glucose` y `BloodPressure`, al ser enteros, se van a imputar por la moda para no obtener decimales.

Primero se van crear los imputadores. Se va a usar el `SimpleImputer` de `scikit-learn`. Para cada tipo de imputador, es decir, por la media o por la moda, se va a crear un imputador. Después se definen los conjuntos de variables que se van a usar en cada imputador. Finalmente, se introducen los imputadores en el `make_column_transformer`.

Como hemos dicho antes, el `make_column_transformer` elimina los atributos que no se le incluyan. Por eso es necesario que las variables `Pregnancies`, `DiabetesPedigreeFunction` y `Age` se tienen que introducir en esta función. Como no es necesario aplicarles ninguna función de transformación, se le pone el parámetro `passthrough`, que indica ésto.

In [ ]:
numerical_imputing = make_pipeline(SimpleImputer(strategy="mean", missing_values=0))
categorical_imputing = make_pipeline(SimpleImputer(strategy="most_frequent", missing_values=0))

num = 'BMI'
cat = 'Glucose|BloodPressure'
non = 'Pregnancies|DiabetesPedigreeFunction|Age'

impute_missing = make_column_transformer(
    (numerical_imputing, make_column_selector(pattern=num)),
    (categorical_imputing, make_column_selector(pattern=cat)),
    ('passthrough', make_column_selector(pattern=non))
)

#### Eliminación de outliers

Para la detección y posterior eliminación de los outliers se va a usar el ensemble `IsolationForest` de `scikit-learn`. Éste es un método no supervisado para identificar los outliers. Cuando se detecte un outlier, se va a borrar toda la instancia.

La eliminación de outliers se realizará después de la imputación de valores perdidos, debido a que es necesario realizar la selección de variables lo primero en el _pipeline_. Además, así evitamos etiquetar como outlier una instancia que por tener valores perdidos se podría clasificar con outlier.

In [ ]:
remove_outliers = FunctionSampler(func=utils.outlier_rejection, kw_args={"random_state":seed})

#### Discretización

Ahora toca el turno de crear un discretizador. En base a las conclusiones obtenidas de la visualización de los datos, podemos observar que partir por igual anchura o igual frecuencia no parece que vaya a obtener buenos resultados. Por esa razón se va a optar por un discretizador por k-medias, siendo k=4.

In [ ]:
discretizer = KBinsDiscretizer(n_bins=4, encode="ordinal", strategy="kmeans")

#### Codificación

Se ha optado realizar la codificación de manera separada a la discretización. El motivo de esta selección ha sido porque facilita la forma de representar el árbol de clasificación. Hemos elegido la codificación *one-hot*. 

In [ ]:
encoder = OneHotEncoder(handle_unknown='ignore')

### Validación de modelos de clasificación

Una vez realizado el preprocesamiento de los datos se va a pasar a la validación de modelos. En este caso se va a realizar la validación de tres modelos diferentes: *Zero-R*, un árbol de decisión sin discretización y un árbol de decisión con discretización.

Para ejecutar los modelos se va a usar un *pipeline*. Se va a usar el pipeline de `imblearn` porque el de `scikit-learn` no permite modificar la variable objetivo al mismo tiempo que las variables predictoras de una instancia. Esto ocurre cuando se eliminan los outliers, que queremos borrar toda la instancia, no solo sus variables predictoras.

Se va a obtener la puntuación que se consigue al hacer *5-cv* solo con el conjunto de entrenamiento, para poder compararlo con la puntuación que se obtiene al usar el conjunto de entrenamiento y el conjunto de test y observar qué sobreajuste se cometería.

#### Algoritmo Zero-R

El primer modelo que vamos a validar es el *Zero-R*. Este modelo sirve como base para base para los demás algoritmos. Para implementarlo se ha usado el `DummyClassifier` de `scikit-learn`. Primero se introducen los pasos del preprocesamiento dentro del *pipeline* junto con el clasificador.

In [ ]:
pipe = make_pipeline(
        impute_missing,
        remove_outliers,
        discretizer,
        encoder,
        DummyClassifier(strategy="most_frequent")
)

Ahora se ejecuta el *pipeline* y obtenemos datos sobre el clasificador.

In [ ]:
predictions, acc, acc_cv, total_time, model = utils.fit_nl_algo(pipe, X_train, y_train, 5, X_test, y_test)

Finalmente, mostramos la matriz de confusión.

In [ ]:
utils.evaluate(model, X_train, X_test, y_train, y_test)

Se puede observar que este clasificador no sirve para este ejemplo. Aunque se obtiene una precisión del 64.94%, que tampoco es demasiado alta, se obtiene un *recall* de 0, es decir, que la tasa de verdaderos positivos es 0. Esto no se puede permitir en un clasificador de diagnóstico, debido a que diagnosticaría que ningún paciente tendría diabetes, y lo que nos importa en este tipo de problemas es detectar a los diabéticos.

También podemos observar que si usamos solo el conjunto de entrenamiento, obtenemos una precisión un poco mayor, es decir, estaríamos sobreajustando un poco.

Además, observamos que se obtiene una puntuación ROC de 0.5, que es la que se espera de un clasificador *Zero-R*.

#### Árbol de decision (Sin discretizar ni codificación)

Tras ejecutar el clasificador *Zero-R* llega el momento de ejecutar el árbol de clasificación. Primero se va a ejecutar el árbol sin discretización. Hay que observar que como no se va a discretizar tampoco hay que codificar las variables. El árbol de clasificación se implementa mediante la clase `DecisionTreeClassifier` de `scikit-learn`. Se va a usar el árbol con máxima profundidad 5, porque se ha observado que es el que mejor generaliza.

In [ ]:
tree_model = DecisionTreeClassifier(random_state=seed, max_depth=5, criterion='entropy')

Creamos el *pipeline*.

In [ ]:
pipe = make_pipeline(
        impute_missing,
        remove_outliers,
        tree_model
)

Ejecutamos el modelo y obtenemos los datos.

In [ ]:
predictions, acc, acc_cv, total_time, model = utils.fit_nl_algo(pipe, X_train, y_train, 5, X_test, y_test)

Finalmente observamos la matriz de confusión que hemos obtenido.

In [ ]:
utils.evaluate(model, X_train, X_test, y_train, y_test)

Aquí podemos observar que se obtiene una precisión del 75% y que es una puntuación más elevada que la obtenida por el *Zero-R*. Es muy importante observar que el *recall* es de 0.64, superando ampliamente al clasificador *Zero-R*, sin embargo este resultado tampoco es un resultado demasiado bueno para un problema de diagnóstico.

Si comparamos la precisión del conjunto de entrenamiento frente al conjunto entero, podemos observar algo curioso, y es que se obtiene una mayor puntuación cuando se realizza con el conjunto de test. Con esto se puede concluir que hemos obtenido un conjunto de test que se adapta muy bien a los datos de entrenamiento. Si hubiéramos cogido otro conjunto de entrenamiento, esto probablemente no pasaría.

Finalmente, si observamos el área bajo la curva ROC, obtenemos un 0.8, un resultado mucho mejor que el *Zero-R*, por lo que podemos concluir que el árbol de decisión sin discretizar es mejor clasificador que el *Zero-R*.

In [ ]:
to_drop_columns=['SkinThickness', 'Insulin']
utils.show_tree(pipe, list(X_train.drop(columns=to_drop_columns).columns), ['0','1'])

Aquí podemos observar el árbol que hemos obtenido. Se ha optado por un árbol de profundidad 5 porque se ha visto que es el que mejor resultado obtenía teniendo en cuenta la profundidad y la complejidad que eso conlleva.

#### Árbol de decision (Con discretizar)

Ahora toca el turno de analizar el árbol de decisión con discretización. En este caso sí que se va a realizar la codificación.

Primero creamos el *pipeline*.

In [ ]:
pipe = make_pipeline(
        impute_missing,
        remove_outliers,
        discretizer,
        encoder,
        tree_model
)

Entrenamos el modelo y obtenemos el resultado.

In [ ]:
predictions, acc, acc_cv, total_time, model = utils.fit_nl_algo(pipe, X_train, y_train, 5, X_test, y_test)

Observamos la matriz de confusión.

In [ ]:
utils.evaluate(model, X_train, X_test, y_train, y_test)

Se puede observar que se obtiene una precisión de un 74.89%, que es inferior a la que hemos obtenido con el árbol sin discretizar. También podemos observar que se obtiene un recall del 0.63, que es inferior al obtenido en el árbol sin discretizar.

Respecto a la comparación de la realización de la evaluación solo con el conjunto de entrenamiento o solo conjunto de test, obtenemos el mismo caso curioso que habíamos observado en el árbol sin discretizar.

Si observamos el área bajo la curva ROC, podemos observar que es de un 0.75, siendo menor que la del árbol sin discretizar, pero mayor que la obtenida con el *Zero_r*.

In [ ]:
to_drop_columns=['SkinThickness', 'Insulin']
utils.show_tree(pipe, pipe[-2].get_feature_names(list(X_train.drop(columns=to_drop_columns).columns)), ['0','1'])

Aquí podemos observar el árbol que se obtiene con discretización y codificación *one-hot*. Podemos observar la codificación en la construcción del árbol en *NombreVariable_num*, siendo *num* un número entre 0 y 3. Esto se debe a la discretización en 4 *bins*.

<br>

**Conclusiones**

Con los resultados que hemos obtenido podemos concluir lo siguiente:

* Si la métrica que nos interesara tener en cuenta fuera la precisión, deberíamos elegir el árbol sin discretizar.

* Si la métrica que nos interesara tener en cuenta fuera el *recall*, deberíamos elegir el árbol sin discretización.

* Si queremos elegir el clasificador que a priori sea el mejor en todos los contextos, deberíamos el elegir el árbol sin discretizar, al ser el que mejor área bajo la curva ROC obtiene.

Como en este caso estamos ante un problema de diagnóstico y nos interesa tener un *recall* alto, el mejor clasificador en este contexto es el árbol sin discretización.

---
---

## Breast cancer wisconsin data

En esta sección de la practica 1 se va a realizar un estudio similar al anterior. En este caso usaremos el dataset de cáncer de pecho en Wisconsin (`breast-cancer-wisconsin-data`).

### Almacenamiento y carga de datos


En este apartado se realizará la carga de datos y su posterior separación en sets de entrenamiento (`train`) y testeo (`test`). También dentro de estos propios conjuntos de datos se separarán los datos en variables (`X`) e instancias o casos (`y`).

Como ya se ha mencionado anteriormente, en esta sección de la práctica se utilizará el dataset: `breast-cancer-wisconsin-data`. Este conjunto de datos esta compuesto por características obtenidas a partir de imágenes digitalizadas de bultos ubicados en el pecho.

In [ ]:
index = "id"
target = "diagnosis" # M = malignant, B = benign

data = utils.load_data("../input/breast-cancer-wisconsin-data/data.csv",index, target)

Dividiremos el conjunto de datos en variables predictoras y variable objetivo (`diagnosis`). Posteriormente separemos los datos, quedándonos con un 75% para la muestra de entrenamiento, y el resto (25%) para la muestra de testeo. 

Para realizar esto se hará uso del método `split_data` del sript `utils`.

In [ ]:
train_size = 0.75
train, tests, X_train, X_test, y_train, y_test = utils.split_data(data, target, seed, train_size)

Mostramos un sample aleatorio de los datos con los que trabajaremos a partir de ahora. (Conjunto `train`)

In [ ]:
train.sample(5, random_state=seed)

El dataset de `Wisconsin` está compuesto por diez variables que a su vez está dividida en otras tres variables. A estas 30 variables hay que sumarle una variable `Unnamed: 32` que no aporta nada al estar vacía y la variable clase `diagnosis` (M = maligno, B = benigno).

Las diez variables predictoras son:
* `radius` Distancia del centro al perímetro
* `texture`: Desviación estándar de los valores en la escala de grises
* `perimeter`: Tamaño del núcleo del tumor
* `area`
* `smoothness`: Variación local en las longitudes de los radios
* `compactness`: Perimetro^2 / área -1
* `concavity`: Gravedad de las porciones cóncavas del contorno
* `concave points`: Número de proporciones cóncavas en el contorno
* `symmetry`
* `fractal dimension`: “coastline approximation” -1

Todas estas variables se subdividen con su correspondiente:
* `mean`: Media del parámetro a medir
* `standard error` (`se`): Standard error de la media (`mean`)
* `worst`: Peor o media de mayor valor de la media

### Análisis exploratorio de datos

Ahora procederemos a realizar un análisis exploratorio de los datos para determinar las propiedades, distribuciones y correlaciones de nuestro conjunto de datos. Usaremos gran cantidad de gráficas y estadísticos para deducir cuáles son las variables predictoras más importantes y cuáles no aportan ninguna información útil en este problema de clasificación.

In [ ]:
train.info(memory_usage=False)

Comenzamos observando la información de cada variable. Obtenemos información como la cantidad de `Non-Null` o el tipo de cada variable. 

De esta información podemos concluir que la variable predictora `Unnamed: 32` no aportará absolutamente nada a nuestro modelo, pues no contiene ni una sola instancia. 

En cuanto al tipo de las variables predictoras, siendo todas estas variables continuas, son de tipo `float64`. Por otra parte, la variable clase al ser un problema de clasificación es una variable discreta de tipo `category`.

In [ ]:
train.shape

Podemos observar que nuestra muestra de entrenamiento cuenta con 426 instancias.

#### Visualización de las variables

Ahora nos vamos a apoyar en representaciones visuales de los datos para ayudarnos a comprender.

Primero vamos a observar si nuestra muestra está balanceada o no. Que como se ve en la gráfica inferior no lo está.


In [ ]:
utils.plot_barplot(train)

In [ ]:
train_mean = train.loc[:,'radius_mean':'fractal_dimension_mean']
train_mean.describe()

In [ ]:
train_se = train.loc[:,'radius_se':'fractal_dimension_se']
train_se.describe()

In [ ]:
train_worst = train.loc[:,'radius_worst':'fractal_dimension_worst']
train_worst.describe()

En las tres tablas superiores, que han sido separadas en `mean`, `se` y `worst`, podemos observar un resumen de los datos estadísticos más importantes. Con esta información podemos darnos cuenta rápidamente que cada variable predictora se mueve en un rango muy diferente de valores. Eso se ha de tener en cuenta en el caso de que se vayan a mostrar dos variables con rangos muy distintos sobre la misma gráfica. O incluso, si usamos técnicas de vecinos más cercanos, pues unas variables tendrían más peso que otras.

También podemos apreciar en la variable `max`, en la mayoría de los casos dista demasiado de la medía y el error. Por lo que es probable que tengamos outliers en nuestros datos. 

##### Distribución de las variables

Antes de pasar a estudiar los posibles outliers de nuestras variables predictoras vamos a estudiar sus distribuciones a ojo:

In [ ]:
utils.plot_histogram(train)

 * `mean`:
  * `radius_mean`: Normal con tendencia a la derecha
  * `texture_mean`: Normal
  * `perimeter_mean`: Normal con tendencia a la derecha
  * `area_mean`: Geométrica o normal con tendencia a la derecha
  * `smoothness_mean`: Normal
  * `compactness_mean`: Geométrica con ruido
  * `concavity_mean`: Exponencial
  * `concave points_mean`: Exponencial
  * `symmetry_mean`: Normal
  * `fractal_dimension_mean`: Normal con tendencia a la derecha
  
  
 * `se`:
  * `radius_se` : Exponencial
  * `texture_se`: Normal con tendencia a la derecha
  * `perimeter_se`: Exponencial
  * `area_se`: Exponencial
  * `smoothness_se`: Normal con tendencia a la derecha
  * `compactness_se`:  Exponencial
  * `concavity_se`: Exponencial o normal con tendencia a la derecha
  * `concave points_se`: Normal
  * `symmetry_se`: Normal con tendencia a la derecha
  * `fractal_dimension_se`: Exponencial
  
  
 * `worst`:
  * `radius_worst`: Mixtura de normales
  * `texture_worst`: Normal
  * `perimeter_worst`: Mixtura de normales
  * `area_worst`: Mixtura de normales o normale en caso de contar con muchos outliers
  * `smoothness_worst`: Normal
  * `compactness_worst`: Normal con tendencia a la derecha
  * `concavity_worst`: Exponencial
  * `concave points_worst`: Normal con tendencia a la derecha
  * `symmetry_worst`: Normal
  * `fractal_dimension_worst`: Normal con tendencia a la derecha


##### Outliers

A continuación, mostraremos los datos usando un diagrama de caja y bigotes _(Boxplot)_. De esta forma podremos observar los outliers por el método de los cuartiles. 

Se ha hecho uso de _dropdown_ para permitir seleccionar la variable que se quiera visualizar. De esta forma nos evitamos tener 32 gráficas diferentes y dejarlo en una sola.

In [ ]:
utils.plot_dropdown_boxplot_ly(train)

En el boxplot superior se pueden observar todas las variables, pudiendo selecionar alguna para ver en concreto.

Por otra parte, se puede apreciar que ciertas distribuciones son considerablemente parecidas. Por ejemplo, `perimeter_mean` y `radious_mean`; `perimeter_se` y `radius_se`. 

Parece indicativo que en nuestro conjunto de datos puede haber variables correlacionadas. Esto lo estudiaremos en el siguiente apartado.

Pero los gráficos de cajas y bigotes no son para observar correlaciones, si no para estudiar outliers. En este caso podemos observar que todas las variables tienen una cantidad de outliers a tener en cuenta en el proceso de preprocesamiento.

##### Correlaciones

Como ya se ha podido intuir en el apartado anterior existen correlaciones entre las variables. Sin embargo, los boxplot no son el tipo de representación más util para ver correlaciones entre variables. Es por eso que se van a usar la función `pairplot` del paquete `seaborn`.

Para poder visualizar de forma correcta estas gráficas se han separado en 3 los datos: diferenciando entre `mean`, `se` y `worst`.

Las conclusiones de las gráficas serán extraidas tras mostrar las tres muestras de datos.

In [ ]:
sns.pairplot(train_mean, kind='reg', plot_kws={'line_kws':{'color':'red'}, 'scatter_kws': {'alpha': 0.5}}, corner=True)

In [ ]:
sns.pairplot(train_se, kind='reg', plot_kws={'line_kws':{'color':'red'}, 'scatter_kws': {'alpha': 0.5}}, corner=True)

In [ ]:
sns.pairplot(train_worst, kind='reg', plot_kws={'line_kws':{'color':'red'}, 'scatter_kws': {'alpha': 0.5}}, corner=True)

Tras mostrar todas las correlaciones podemos observar algunas relaciones claras como son el caso de `area`, `perimeter` y `radius` en los tres grupos de `mean`, `se` y `worst`. Otras correlaciones tienen mayor dispersion como serian `compactness` y `fractal_dimension`. Estas correlaciones que se repiten en las tres muestras por las que se han mostrado los datos nos hace pensar que incluso entre ella existan correlaciones.

___


Tras realizar un análisis para los conjuntos `mean`, `se`, `worst` para buscar correlaciones entre variables predictoras, ahora tocará buscar correlaciones entre estos tres conjuntos.

Para realizar este estudio en busca de las correlaciones usaremos un mapa de calor con la que obtendremos de forma numérica y visual todas las correlaciones entre todas las variables. Siendo 1 una correlación directa absoluta, 0 ninguna correlación y -1 una correlación inversa.

In [ ]:
utils.corr_heatmap_ly(train, height=800)

Con nuestra matriz de confusión representada con en un heatmap podemos observar de un solo vistazo la gran cantidad de variables relacionadas, incluso entre los conjuntos `mean`, `se` y `worst`.

Empecemos analizando las variables predictoras del conjunto `worst`. En estas variables se puede ver una tendencia en su correlación con la muestra de `mean`. Esto se debe a por la propia naturaleza del problema, `worst` es un subconjunto de datos de `mean`.

A continuación, nos centraremos en las variables predictoras `radius`, `perimeter` y `area` tanto del conjunto `mean` como del conjunto `se`. Por definición, el área y el perímetro se pueden obtener apartir del radio y eso se ve reflejado en nuestros datos de entrenamiento. Por lo tanto, `perimeter` y `area` se deberían poder eliminar sin problema alguno.

Si procedemos igual que en el caso anterior y observamos `concavity`, `concave points` y `compactness`, seleccionaremos `concave points` eliminando las otras.

En la nueva heatmap inferior se observa las relaciones entre variables predictoras si se eliminarán las variables mencionadas.

In [ ]:
utils.corr_heatmap_ly(train.drop(columns=list(train_worst.columns)+['Unnamed: 32','perimeter_mean','perimeter_se','area_mean','area_se','concave points_mean','concave points_se','compactness_mean','compactness_se']))

##### Poder discriminatorio

Una vez ya hemos observado todas las correlaciones entre todas las propias variables predictoras, obtenemos información que nos será muy útil para poder eliminar variables ya que no aportarían información. Ahora vamos a estudiar las relaciones entre las variables predictoras y las variables clase, es decir, vamos a medir el poder predictivo de todas las variables mediante la observación de gráficas de distribución condicionada.

Para buscar estas correlaciones de manera visual vamos a usar las gráficas tipo `histogram` proporcionadas por la librería `plotly`. Sin embargo, se han realizado modificaciones sobre dicha gráfica para permitir un _dropdown_ como en el caso de los _outliers_ y con una sola gráfica poder seleccionar y observar todas las variables. 

In [ ]:
utils.plot_dropdown_distplot_ly(train, y_train)

Con estos histogramas pretendemos observar el poder predictivo de cada variable. Esto se puede observar de forma muy visual pues aquellas gráficas que tengan mayor área superpuesta entre las dos variables clase `diagnosis` serán las que menor información aporten a la hora de discretizar. Y en caso contrario, a menor área superpuesta, mayor poder predictivo.

A continuación, se analizará su aparente poder predictivo:

* `perimeter_mean`, `area_mean`, `radius_mean`, `concavity_mean` y `concave points_mean`: Muy poco área conjunta. Aparentemente gran poder predictivo.
* `texture_mean` y `compatness_mean`: Sin ser las variables predictoras más significativas pueden aportar información.
* `smoothness_mean`, `symmetry_mean` y `fractal_dimension_mean`: Capacidad de discretización muy limitado por sí solas.


* `perimeter_se`, `area_se` y `radius_se`: Muy poco área conjunta. Aparentemente gran poder predictivo.
* `concave points_se` y `concavity_se`: Sin ser las variables predictoras más significativas pueden aportar información.
* `compatness_se`, `texture_se`, `smoothness_se`, `symmetry_se` y `fractal_dimension_se`: Capacidad de discretizacion muy limitado por sí solas.


* `perimeter_worst`, `concavity_worst`, `concave points_worst`, `area_worst` y `radius_worst`: Muy poco área conjunta. Aparentemente gran poder predictivo.
* `texture_worst` y `compatness_worst`: Sin ser las variables predictoras más significativas pueden aportar información.
* `smoothness_worst`, `symmetry_worst` y `fractal_dimension_worst`: Capacidad de discretizacion muy limitado por sí solas.

##### Conclusiones

Tras haber realizado las visualizaciones oportunas en busca de outliers, correlaciones entre variables predictoras y el valor de cada variable predictora que puede aportar al modelo hemos sacado unas conclusiones a tener en cuenta en la siguiente fase, la de preprocesamiento.

* Se han detectado outliers que han de ser eliminados o imputados.
* Hay variables fuertemente relacionadas que se podrán eliminar.
* Existen variables que proporcionaran mayor poder predictivo. 

### Preprocesamiento de Datos
Una vez hemos realizado el análisis exploratorio y acabar de ver las conclusiones que hemos obtenido del mismo debemos moldear nuestros datos acordemente. Aquí es donde entra en juego la parte del preprocesamiento. En nuestro conjunto de datos nos centraremos en la corrección de los outliers así como en la selección de variables. 

Cabe destacar que todos los métodos que se van a usar, tanto los importados como los propios, tienen el objeto de funcionar dentro de un `Pipeline` de la librería de `imblearn` posteriormente. 

#### Selección de variables

Empezaremos realizando una selección de variables predictoras. Nos basaremos en las correlaciones obtenidas y en el poder predictivo que hemos observado en el análisis exploratorio de los datos. 

Cabe señalar que dado que hay una variable predictora completamente vacía (`Unnamed: 32`) se eliminará directamente, pues no hay posibilidad de imputarla. No habrá que preocuparse de imputar más instancias pues ya hemos comprobado que no hay datos perdidos.

La selección de variables será el primer paso que se realizará dado que de esta forma reducimos de forma considerable el coste computacional de las posteriores operaciones a realizar.

In [ ]:
# Variables con fuertes correlaciones
correlations_worst = list(train_worst.columns)
correlations = [
    'perimeter_mean',
    'perimeter_se',
    'area_mean',
    'area_se',
    'concavity_mean',
    'concavity_se',
    'compactness_mean',
    'compactness_se'
]

# Con nulo poder predictivo 
sin_predict = [
    'texture_se',
    'smoothness_se',
    'symmetry_se',
    'fractal_dimension_se',
    'smoothness_mean',
    'symmetry_mean',
    'fractal_dimension_mean'
]

to_drop_columns = ['Unnamed: 32'] + correlations_worst + correlations + sin_predict

Tras _droppear_ todas estas variables se nos quedaría un conjunto de entrenamiento como el siguiente:

In [ ]:
dropped_data = train.drop(columns=to_drop_columns)
dropped_data.sample(5, random_state=seed)

El método que usará el Pipeline es un método propio implementado en el utils.

In [ ]:
drop_transformer = utils.drop_columns(colnames=to_drop_columns)

Por último, hay que destacar que esta selección de variables ha sido escogida con el simple criterio de la lógica y observación de gráficas ya que aún no han sido explicados los métodos de selección de variables en teoría.

#### Tratamiento de outliers

Para la localización y tratamiento de outliers se ha optado por hacer uso del ensemble `IsolationForest` implementado en la librería de sklearn. Este algoritmo ensemble se encarga de buscar anomalías en los datos por medio de estructuras en forma de árbol.

Los outliers son tratados eliminando la instancia completamente.

Con el objeto de que los casos sean reproducibles le tendremos que pasar como hiperparametro la semilla `seed`.

In [ ]:
remove_outliers = FunctionSampler(func=utils.outlier_rejection, kw_args={"random_state":seed})

Este proceso se hace justo después de la eliminación de las variables predictoras, pues el algoritmo usado para el tratamiento de outliers `IsolationForest` monta el ensemble haciendo uso de todas las variables predictoras que se le aporten. Esto quiere decir, que no corregirá el mismo numero de outliers con las variables seleccionadas que sin seleccionar. 

In [ ]:
# Eliminacion de outliers con todas las variables predictoras excepto Unnamed: 32
print(f"Numero instancias *sin* seleccion de variables: {remove_outliers.fit_resample(X_train.drop(columns=['Unnamed: 32']),y_train)[0].shape[0]}")

# Eliminacion de outliers con la seleccion de variables realizada
print(f"Numero instancias *con* seleccion de variables: {remove_outliers.fit_resample(dropped_data.iloc[:,:-1],y_train)[0].shape[0]}")

#### Discretización 

Con el objeto de obtener unos datos más simples y convertir las variables numéricas en intervalos vamos a discretizar esas variables, siendo en este caso, todas las variables predictoras.

Se ha optado por una discretización con 4 secciones que nos ha dado los mejores resultados y no añadía excesiva complejidad al modelo. La estrategia de discretización seleccionada es por igual frecuencia.

Para no alargar la libreta más de lo necesario se han omitido todas las iteraciones buscando la mejor configuración para el discretizador y los procesos de preprocesamiento. Por lo tanto, en esta libreta solo se mostrará el modelo más simple y con mejor resultado que hemos sido capaces de obtener. 

In [ ]:
discretizer = KBinsDiscretizer(n_bins=4, encode="ordinal", strategy="quantile")

#### Codificación

Dado que nuestro objetivo, aparte de crear el mejor modelo posible, también es crear un modelo que sea sencillo y legible posible hemos optado por codificar los datos. Este paso no es imprescindible para los modelos que vamos a probar (Zero-R y árboles de decisión) pues son modelos capaces de trabajar con valores numéricos. Sin embargo, y motivados por un modelo legible, vamos a codificar los datos con la codificación `OneHot` haciendo uso del método proporcionado por `sklearn`.

In [ ]:
encoder = OneHotEncoder(handle_unknown='ignore')

### Validación de modelos de clasificación

En este apartado vamos a realizar la validación de diferentes modelos de clasificación. Para este apartado solo contamos con los algoritmos *Zero-R*, que usaremos de base-line; y dos árboles de clasificación, uno con discretización y codificación *one-hot* y el otro árbol sin esto.

Recordar que en esta libreta no se busca obtener los mejores hiperparámetros por los métodos de selección de modelos. Y por lo tanto se han elegido dichos hiperparámetros en base a unas observaciones de las gráficas obtenidas en apartados anteriores.

Para que estos experimentos sean reproducibles se hará uso siempre que sea necesario del hiperparámetro `random_state` que nos permite controlar las computaciones de eventos aleatorios.

Por comodidad se usará un `pipeline` que aplicará todos los preprocesamientos y clasificará según los métodos que le pasemos. Hacer uso de este pipeline también nos garantizará que no se produzca un *data-leak*.

#### Algoritmo Zero-R

Primero de todo crearemos y analizaremos un _Zero-R_. Un algoritmo trivial que clasificara siempre la clase más predominante. Este algoritmo nos servirá de base-line. Esto quiere decir que los futuros algoritmos que analicemos tienen que tener **siempre** mejores resultados.

Primero crearemos su correspondiente Pipeline.

In [ ]:
pipe = make_pipeline(
        drop_transformer,
        remove_outliers,
        discretizer,
        encoder,
        DummyClassifier(strategy="most_frequent")
)

In [ ]:
predictions, acc, acc_cv, total_time, model = utils.fit_nl_algo(pipe, X_train, y_train, 5, X_test, y_test)

Podemos observar que las puntuaciones obtenidas son bastante malas (A excepcion del _recall_ de `B`).  Aunque en este caso es tontería hacer una validación cruzada pues siempre obtendremos el mismo resultado. Sin embargo, antes de entrar en evaluar los resultados tenemos que explicar que representa cada valor:

Se han usado dos validaciones. Una usando el conjunto de entrenamiento (únicamente la validacion cruzada) y otra usando el conjunto de test. 

* `train`:
 * `Model cv score`: Validación cruzada
* `test`:
 * `Model regular score`: Precisión estándar del modelo
 * `Model time`: Tiempo de entrenamiento más validación cruzada
 * `precisión`: Probabilidad de no clasificar como positivo un ejemplo negativo
 * `recall`: Probabilidad de encontrar los casos positivos 
 * `f1-score`: Media de `precisión` y `recall`
 * `support`: Números de casos 
 * `ROC`: Área bajo la curva

Tomaremos estos datos como base-line

In [ ]:
utils.evaluate(model, X_train, X_test, y_train, y_test)

En la matriz de confusión superior se puede observar de forma muy gráfica el funcionamiento del algoritmo. Clasificará todo como `B`. Por lo tanto, acertará todos los casos `B` pero fallará todos los casos `M`.

#### Árbol de decision (Sin discretizar ni codificación)

Una vez hemos observado los datos que obtenemos realizando un *Zero-R* y usaremos como base-line podemos proseguir con modelos más complejos. Ahora haremos un árbol de decisión. Sin embargo, no le aplicaremos el paso de discretizar y codificación para observar cómo puede afectar a nuestro modelo.

In [ ]:
tree_model = DecisionTreeClassifier(random_state=seed, max_depth=3, criterion='entropy')

Como hiperparámetros del árbol hemos optado por una profundidad máxima de 3, con el objeto de que el árbol no ramifique demasiado y por consiguiente sobreajuste a los datos de entrenamiento. Si se dejara crecer el árbol infinitamente crecería hasta una profundidad de 7, profundidad donde se realizaría un sobreajuste prácticamente total.

También se usará el criterio de la entropía para crear las particiones.

Haremos uso del pipeline igual que en el caso anterior. Y repetiremos el proceso para la obtención de los datos de medición. Los evaluaremos posteriormente.

In [ ]:
# Pipeline
pipe = make_pipeline(
        drop_transformer,
        remove_outliers,
        tree_model
)

In [ ]:
# Fit del modelo y muestra resultados
predictions, acc, acc_cv, total_time, model = utils.fit_nl_algo(pipe, X_train, y_train, 5, X_test, y_test)

In [ ]:
utils.evaluate(model, X_train, X_test, y_train, y_test)

Los resultados obtenidos con la métrica de la validación cruzada y precisión son bastante buenos. Sin embargo, dada la naturaleza del problema no nos interesa que nuestro modelo erre al predecir los casos malignos `M`. Este es el problema de este modelo. 

Si nos fijamos en el `recall` de `M` y de igual manera se puede observar en la matriz de confusión, predecir casos malignos tiene una probabilidad de encontrar los casos positivos de un 81%. Esto quiere decir que tendremos un 19% de falsos negativos, y en este problema puede ser gente que pierda la vida. Por lo tanto, a pesar de tener un buen funcionamiento general debemos refinar los falsos negativos para la variable `M`.


A continuacion se muestra una representacion gráfica del arbol.

In [ ]:
utils.show_tree(pipe, list(X_train.drop(columns=to_drop_columns).columns), ['B','M'])

---

#### Árbol de decision (Con discretizar)

Para finalizar en la búsqueda del mejor modelo vamos a repetir el mismo proceso anterior, pero en este caso aplicaremos la discretización y codificación vistas en el apartado del preprocesamiento.

In [ ]:
# Pipeline
pipe = make_pipeline(
        drop_transformer,
        remove_outliers,
        discretizer,
        encoder,
        tree_model
)

In [ ]:
# Fit del modelo y muestra resultados
predictions, acc, acc_cv, total_time, model = utils.fit_nl_algo(pipe, X_train, y_train, 5, X_test, y_test)

In [ ]:
utils.evaluate(model, X_train, X_test, y_train, y_test)

Tras aplicar la discretización y la codificación podemos no observar grandes cambios en la puntuación por evaluación cruzada, incrementándose apenas un 1%. Por otra parte, sí se observa un aumento mayor del área bajo la curva.

También se puede apreciar que el `recall` de `M` ha aumentado, mientras que el de `B` ha disminuido. Sin embargo, esto es lo que se buscaba. Una reducción de los falsos negativos. Siendo una puntuación mejorable, pues de cada 10 pacientes con un tumor maligno, 1 no lo detectas, aunque está mejor que los modelos anteriores.

A continuación se muestra una representación gráfica del árbol.

In [ ]:
utils.show_tree(pipe, pipe[-2].get_feature_names(list(X_train.drop(columns=to_drop_columns).columns)), ['B','M'])

Tras haber realizado la validación podemos observar que obtenemos resultados muy similares a los obtenidos en la fase de entrenamiento donde validábamos con el conjunto de entrenamiento.

Una precisión del 92.31%, un área bajo la curva del 92.83% y un recall del 94% y 89% para `B` y `M` respectivamente es lo que obtenemos con el conjunto de pruebas.

<br>

---
<br>

**Conclusiónes**

Con un modelo tremendamente sencillo, que tan solo hace uso de tres variables predictoras (`concave points_mean`, `radius_mean`, `texture_mean`) como se puede ver en la representación gráfica del árbol, tenemos un modelo predictivo, que sin ser el mejor, nos otorga unas predicciones precisas en un 92%.

También habrá que tener en cuenta que al requerir tan pocas variables predictoras el modelo final, el usuario que lo use no tendrá que tomar las 30 medidas iniciales que había en la base de entrenamiento inicial.

Por lo tanto, este modelo no se puede usar como la única fuente de predicción pero un experto sí se podría basar en ella para validar sus diagnosticos. 
